## Dataset Pre-processing
### Download Beauty Dataset from [Amazon 2018](https://nijianmo.github.io/amazon/index.html)

In [1]:
!python data/data_process.py --file_path data/Beauty/All_Beauty_5.json.gz --output_path data/Beauty

python: can't open file 'data/data_process.py': [Errno 2] No such file or directory


## Load Dependencies and Hyper-Parameters

In [2]:
import torch

import sys
sys.path.append('../')

from src.models import model_factory
from src.dataloaders import dataloader_factory
from src.datasets import dataset_factory
from src.trainers import trainer_factory
from src.utils.utils import *
from src.utils.options import parser

In [16]:
args = parser.parse_args([])

args.data_path = '../data/ml-1m' # '../data/Beauty'
args.num_epochs = 100
args.trm_max_len = 50

ckpt_root = setup_train(args)

Folder created: /home/zhankui/1_engineering/Qualcomm-FastRec/demo/experiments/test_2022-01-04_18
{'adam_epsilon': 1e-06,
 'best_metric': 'NDCG@10',
 'data_path': '../data/ml-1m',
 'dataloader_code': 'sasrec',
 'dataloader_random_seed': 0.0,
 'dataset_code': 'item',
 'dataset_split_seed': 98765,
 'device': 'cpu',
 'device_idx': '0',
 'experiment_description': 'test',
 'experiment_dir': 'experiments',
 'global_epochs': 1000,
 'local_epochs': 10,
 'lr': 0.001,
 'metric_ks': [5,
               10,
               20],
 'model_code': 'sasrec',
 'model_init_seed': 0,
 'num_epochs': 100,
 'num_gpu': 1,
 'optimizer': 'Adam',
 'split': 'leave_one_out',
 'subset_size': 1000,
 'test_batch_size': 64,
 'test_negative_sample_size': 100,
 'test_negative_sampler_code': 'random',
 'test_negative_sampling_seed': 98765,
 'train_batch_size': 64,
 'train_negative_sample_size': 100,
 'train_negative_sampler_code': 'random',
 'train_negative_sampling_seed': 0,
 'trainer_code': 'sasrec_sample',
 'trm_att_dropo

## Build Dataset

In [17]:
dataset = dataset_factory(args)
train_loader, val_loader, test_loader, dataset = dataloader_factory(args, dataset)

## Model Setup
We load [SASRec Model](https://arxiv.org/abs/1808.09781) for sequential recommendation

In [18]:
model = model_factory(args)
if args.load_pretrained_weights is not None:
    print("weights loading from %s ..." % args.load_pretrained_weights)
    model = load_pretrained_weights(model, args.load_pretrained_weights)
print("Model size:", sum(p.numel() for p in model.parameters() if p.requires_grad))
print(model)

Model size: 204500
SASRecModel(
  (loss): BCEWithLogitsLoss()
  (item_emb): Embedding(3418, 50, padding_idx=3417)
  (pos_emb): Embedding(50, 50)
  (emb_dropout): Dropout(p=0.2, inplace=False)
  (attention_layernorms): ModuleList(
    (0): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
    (1): LayerNorm((50,), eps=1e-08, elementwise_affine=True)
  )
  (attention_layers): ModuleList(
    (0): MultiHeadAttention(
      (linear_layers): ModuleList(
        (0): Linear(in_features=50, out_features=50, bias=True)
        (1): Linear(in_features=50, out_features=50, bias=True)
        (2): Linear(in_features=50, out_features=50, bias=True)
      )
      (output_linear): Linear(in_features=50, out_features=50, bias=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (1): MultiHeadAttention(
      (linear_layers): ModuleList(
        (0): Linear(in_features=50, out_features=50, bias=True)
        (1): Linear(in_features=50, out_features=50, bias=True)
        (2): Linear(in_fe

## Model Training

In [19]:
trainer = trainer_factory(args, model, train_loader, val_loader, test_loader, ckpt_root, dataset.data)
trainer.train()

TEST N@5 0.02862: , N@10 0.04380: , N@20 0.06933: , R@5 0.04885: , R@10 0.09671: , R@20 0.19863: , M 0.05117: , AUC 0.50692: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 123.48it/s]
  0%|          | 0/95 [00:00<?, ?it/s]../src/trainers/base.py:85: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.model.parameters(), 5.0)
Epoch 10, loss 0.31915 : 100%|██████████| 95/95 [00:04<00:00, 20.37it/s]
VAL N@5 0.31697: , N@10 0.36441: , N@20 0.40247: , R@5 0.44868: , R@10 0.59567: , R@20 0.74567: , M 0.31037: , AUC 0.84982: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 165.32it/s]
TEST N@5 0.31400: , N@10 0.36027: , N@20 0.40209: , R@5 0.43973: , R@10 0.58259: , R@20 0.74833: , M 0.30986: , AUC 0.84951: , loss 0.00000:  15%|█▍        | 14/95 [00:00<00:00, 138.29it/s]

Update Best NDCG@10 Model


TEST N@5 0.30572: , N@10 0.35232: , N@20 0.39109: , R@5 0.43196: , R@10 0.57582: , R@20 0.72873: , M 0.30133: , AUC 0.84338: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 168.26it/s]
Epoch 20, loss 0.18691 : 100%|██████████| 95/95 [00:04<00:00, 20.36it/s]
VAL N@5 0.36025: , N@10 0.40325: , N@20 0.43799: , R@5 0.49490: , R@10 0.62834: , R@20 0.76557: , M 0.34906: , AUC 0.85752: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 148.10it/s]
TEST N@5 0.35145: , N@10 0.39611: , N@20 0.42792: , R@5 0.48493: , R@10 0.62388: , R@20 0.74944: , M 0.34077: , AUC 0.85584: , loss 0.00000:  17%|█▋        | 16/95 [00:00<00:00, 159.08it/s]

Update Best NDCG@10 Model


TEST N@5 0.34673: , N@10 0.39025: , N@20 0.42388: , R@5 0.47648: , R@10 0.61064: , R@20 0.74386: , M 0.33774: , AUC 0.84931: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 146.09it/s]
Epoch 30, loss 0.13565 : 100%|██████████| 95/95 [00:04<00:00, 20.37it/s]
VAL N@5 0.37683: , N@10 0.42121: , N@20 0.45387: , R@5 0.50499: , R@10 0.64221: , R@20 0.77083: , M 0.36778: , AUC 0.86258: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 161.37it/s]
TEST N@5 0.38187: , N@10 0.42409: , N@20 0.45517: , R@5 0.50857: , R@10 0.63911: , R@20 0.76159: , M 0.37219: , AUC 0.85605: , loss 0.00000:  17%|█▋        | 16/95 [00:00<00:00, 158.67it/s]

...

TEST N@5 0.36508: , N@10 0.40807: , N@20 0.43920: , R@5 0.48942: , R@10 0.62231: , R@20 0.74545: , M 0.35694: , AUC 0.85303: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 165.83it/s]
Epoch 70, loss 0.06502 : 100%|██████████| 95/95 [00:05<00:00, 17.77it/s]
VAL N@5 0.38391: , N@10 0.42600: , N@20 0.45753: , R@5 0.51245: , R@10 0.64172: , R@20 0.76612: , M 0.37377: , AUC 0.86184: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 138.96it/s]
TEST N@5 0.36475: , N@10 0.40615: , N@20 0.43861: , R@5 0.48980: , R@10 0.61683: , R@20 0.74485: , M 0.35636: , AUC 0.85208: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 153.96it/s]
Epoch 80, loss 0.05901 : 100%|██████████| 95/95 [00:04<00:00, 20.26it/s]
VAL N@5 0.38532: , N@10 0.42732: , N@20 0.45922: , R@5 0.51425: , R@10 0.64375: , R@20 0.77012: , M 0.37474: , AUC 0.86207: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 161.00it/s]
TEST N@5 0.33562: , N@10 0.38117: , N@20 0.41273: , R@5 0.45544: , R@10 0.59606: , R@20 0.72106:

Update Best NDCG@10 Model


TEST N@5 0.36561: , N@10 0.40748: , N@20 0.43781: , R@5 0.49178: , R@10 0.62100: , R@20 0.74090: , M 0.35634: , AUC 0.85109: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 137.55it/s]
Epoch 90, loss 0.05367 : 100%|██████████| 95/95 [00:04<00:00, 20.22it/s]
VAL N@5 0.38730: , N@10 0.42856: , N@20 0.45967: , R@5 0.51985: , R@10 0.64715: , R@20 0.77018: , M 0.37516: , AUC 0.86425: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 153.96it/s]
TEST N@5 0.37776: , N@10 0.42187: , N@20 0.45012: , R@5 0.50460: , R@10 0.63971: , R@20 0.75138: , M 0.36831: , AUC 0.85496: , loss 0.00000:  19%|█▉        | 18/95 [00:00<00:00, 171.27it/s]

Update Best NDCG@10 Model


TEST N@5 0.36802: , N@10 0.40971: , N@20 0.44109: , R@5 0.49550: , R@10 0.62352: , R@20 0.74797: , M 0.35843: , AUC 0.85403: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 169.82it/s]
Epoch 100, loss 0.05043 : 100%|██████████| 95/95 [00:04<00:00, 20.13it/s]
VAL N@5 0.38439: , N@10 0.42627: , N@20 0.45667: , R@5 0.51732: , R@10 0.64644: , R@20 0.76656: , M 0.37237: , AUC 0.86318: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 158.60it/s]
TEST N@5 0.36579: , N@10 0.40814: , N@20 0.43912: , R@5 0.49073: , R@10 0.62133: , R@20 0.74359: , M 0.35732: , AUC 0.85301: , loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 161.86it/s]


## Model Test and Saving

In [20]:
# model testing and saving
trainer.test()
trainer.logger_service.complete({'state_dict': (trainer._create_state_dict())})
trainer.writer.close()

FINAL TEST: N@5 0.38049, N@10 0.42156, N@20 0.45156, R@5 0.50541, R@10 0.63161, R@20 0.75060, M 0.37080, AUC 0.85384, loss 0.00000:  15%|█▍        | 14/95 [00:00<00:00, 136.33it/s]

Test best model with test set!


FINAL TEST: N@5 0.36755, N@10 0.40923, N@20 0.44075, R@5 0.49496, R@10 0.62297, R@20 0.74797, M 0.35800, AUC 0.85377, loss 0.00000: 100%|██████████| 95/95 [00:00<00:00, 142.63it/s]

{'NDCG@5': 0.3675452088054858, 'Recall@5': 0.4949561401417381, 'NDCG@10': 0.409231195010637, 'Recall@10': 0.6229714914372093, 'NDCG@20': 0.44075291501848324, 'Recall@20': 0.7479714914372093, 'MRR': 0.3580012647729171, 'AUC': 0.8537730348737617}


## Export ONNX Model 
- Install ONNX with `pip install ONNX`
- Install ONNX Runtime ()ORT with `pip install onnxruntime` or `pip install onnxruntime-gpu`

In [21]:
model.eval()
for batch in trainer.test_loader:
    break
users, seqs, candidates, labels, length = batch
inputs = (seqs, candidates, length)
torch_out = model(*inputs)

### Export Our Model with ONNX (Ver. 11)

In [22]:
torch.onnx.export(model,               # model being run
                  inputs,                         # model input (or a tuple for multiple inputs)
                  "model.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['seqs', 'candidates', 'length'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

/home/zhankui/.local/lib/python3.6/site-packages/torch/onnx/utils.py:1190: UserWarning: Provided key input for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))
../src/models/onnx_support.py:74: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(query.size(-1))
/home/zhankui/.local/lib/python3.6/site-packages/torch/onnx/symbolic_opset9.py:2624: UserWarning: Exporting aten::index operator of advanced indexing in opset 11 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  "If indices

### Load Our ONNX Model for Checking

In [10]:
import onnx

onnx_model = onnx.load("model.onnx")
onnx.checker.check_model(onnx_model)

### Use ONNXRuntime to Run our Model

In [11]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("model.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

/home/zhankui/anaconda3/envs/bert4rec/lib/python3.6/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:353: UserWarning: Deprecation warning. This ORT build has ['CUDAExecutionProvider', 'CPUExecutionProvider'] enabled. The next release (ORT 1.10) will require explicitly setting the providers parameter (as opposed to the current behavior of providers getting set/registered by default based on the build flags) when instantiating InferenceSession.For example, onnxruntime.InferenceSession(..., providers=["CUDAExecutionProvider"], ...)
  "based on the build flags) when instantiating InferenceSession."


In [12]:
# compute ONNX Runtime output prediction
ort_inputs = {'seqs': to_numpy(seqs), 'candidates': to_numpy(candidates), 'length': to_numpy(length)}
ort_outs = ort_session.run(None, ort_inputs)

In [13]:
# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

### You can also use `onnx_tensorrt` to use `TensorRT` backend

- Here we use TensorRT 8.2.0, please follow [onnx_tensorrt](https://github.com/onnx/onnx-tensorrt) master branch to install the related dependencies. Or you can use `onnx2trt` to export this `.onnx` to `.trt` engine.

In [ ]:
import onnx
import onnx_tensorrt.backend as backend

onnx_model = onnx.load("model.onnx")
engine = backend.prepare(onnx_model, device='CUDA:0')

## Model Serving
We deploy a model with [faiss](https://github.com/facebookresearch/faiss) to accelerate the maximal inner product search. 

In [ ]:
import os
import faiss

model = model_factory(args)
model = load_pretrained_weights(model, os.path.join('../', ckpt_root, 'models', 'best_acc_model.pth'))
model = model.eval()

In [15]:
import numpy as np
d = args.trm_hidden_dim                                 # dimension
nb = model.item_emb.weight.size(0)                      # item pool size
xb = model.item_emb.weight.data.cpu().numpy()           # item embeddings table

In [16]:
index = faiss.index_factory(d, 'HNSW32', faiss.METRIC_INNER_PRODUCT)   # build the index
index.add(xb)                                                          # add vectors to the index

### Model Input Example

In [41]:
x = torch.LongTensor([dataset.train[0]])                # input example
l = torch.LongTensor([len(dataset.train[0])-1])         # length

In [23]:
xq = model(x, length=l, mode='serving').detach().cpu()

### Exact Search via Naive Matmul

In [34]:
k = 10

V, I = torch.topk((torch.Tensor(xb) @ xq.squeeze()), k)
print(I)

tensor([63, 29, 28, 61, 36,  6, 49, 62, 12, 39])


### Approximate Search Via Faiss

In [40]:
V, I = index.search(xq.numpy(), k)
print(I)

[[63 29 28 36 61  6 62 12 49 39]]
